# Assignment 1 - SQL, NoSQL and Data Warehouses
---

##### The Tutorial will take place online on Friday, Nov 27th 2020 at 11:00am (CET) c.t. <br><br>This Assignment is due on Monday, Dec 7th 2020 at 11:59pm (CET)<br>The next Assignment (Assignment 2) will be published on Dec 15th (so there is a one week break)!

| Total Points | Points Needed To Pass (50%) | Points Reached |
|:-:|:-:|:-----------:|
| 10 | 5 |  |

Hello there! This is the first (actual) Assignment for Big Data Management and Analytics this year. Here, we will take a practical look at databases as well as data warehouses. In this assignment you will also find the first graded <font color='#782769'>TODOs</font>, which are <font color='#e52425'>clearly marked</font> and are mandatory to submit to take part in the exam, so please read the assignment carefully. 

## Goals

In this assignment you will work at a fictonal shop, selling bikes via a website. Your boss first tasks you with setting up a database for all your stored information but as bosses do, he might change his mind about what we wants a couple of times on the way. At the end of this Assignment you should have a better understanding of how different structures of databases work and what their respective shortcomings or strenghts are.

## Step 0 - Setup

Because we are mainly working on databases now, you need some software to manage and set up a database on your local PC. For this we will use PostgreSQL (often pronounced "Postgres"). PostgreSQL is a free, open-source database management tool that you can download from [their website](https://www.postgresql.org/download/).<br>
After downloading please install it and its necessary components. **During the installation you will be asked to set a master password, make sure to remember it**. When the Setup is complete you can open "pgAdmin 4", which opens up your browser with a window similar to this:
<img src="media/Screenshot_pgAdmin.png" height=600>
Now, open up your PostgreSQL 13 Server with the small arrow next to "Servers" until you can see the "postgres" sample database. Right-click on "Databases" to create a new one, we will call "bikeDB". If you click on it, your left side should look something like this:
<img src="media/Screenshot_pgAdmin1.png" height=600>
Next, right-click again onto your "bikeDB" and choose "CREATE Script" from the menu. You should now see a workspace similar to this:
<img src="media/Screenshot_pgAdmin2.png" height=600>
You are now ready to go to work on your (yet to be filled) database.

## Step 1 - Creating a Relational Database

Imagine, you are working for a company that sells bikes online. You are new at the company but has been tasked with setting up a database to gain more insights about the company's recent sales. <br> 
The first thing you need to do is to get familiar with the data as well as the relations between the different entities. <br>*(As we hope, you have some experience with SQL and databases in general, we will not go into detail here but we will do a recap in the Tutorial, so watch the recording if you missed it and need a refresh!)* <br> One of your collegues can provide you with an Entity Relationship Diagram to get a first overview:
<img src="media/ER.png" width=600 height=600>
Lucky for you, the scripts for creating the tables can still be found somewhere, so you don't have to start from the scratch at least. ;)

In pgAdmin, use the "Open File" button in the top left corner (below "Dashboard") to open up the **CREATEbikeDB.sql** file into your Query Editor (you can also drag and drop it). Take a closer look at the first lines, especially the CREATE TABLE statements to get familiar with datatypes and again the overall structure of the database. <br> Then execute the query with F5 or the button on top. You should get a few warnings but other than that, your database should be good to go!

## Step 2 - Including Data from other Files

First we want to make sure that all tables are present and all data was inserted correctly. For this we are gonna start with a simple query that will return the first $n$ entries of a table. In pgAdmin open the "query tool", that can be found in the top left corner (below Dashboard) or by clicking on "Query Tool" in the "Tools" menu section on top. You should then get an empty window, where you can type your SQL Query and again execute it by pressing F5 (or the button).

```sql
--- This is a simple SQL statement to get the first 30 entries from the table article
--  This will not execute here in Jupyter but you will need to copy it into pgAdmin 4

SELECT * FROM public.article -- public.orders, public.orderposition, public.articlereservation
ORDER BY 1 ASC 
LIMIT 30;
```

Feel free to try out some more small queries until you get a good feel of the data. <br><br>
You then present the database to your boss to what he tells you that there is more data, that he would like to include. This new data was not stored in a database before, so the data is stored in csv, json and xml files. Now your boss wants you to include these files into your existing database.

### <font color='#782769'>Now it's your turn!<br> TODO #1:</font>

Preprocess the files before insertion into the tables Orders, Orderposition and Articlereservation

Write a small python script which will parse and include the data stored in three different kind of files all located in the folder **data_to_stage** (articlereservation.csv, ORDER.csv, ORDER_10.json to ORDER_13.json and ORDER_1.xml to ORDER_9.xml - *total of 15 files*) into your bikeDB.

Make sure you pay attention to the relations within the data, so you create no open and dangling references! *(Hint: You should first insert into Orders)*

In [3]:
## Check which files need to be staged!
import os
import pandas as pd
from datetime import datetime as dt
import json
from functools import reduce
from parse import *
from pandas.io.json import json_normalize
import pandas_read_xml as pdx
from pandas_read_xml import auto_separate_tables

file_name = os.listdir('./data_to_stage')
csv_df = list()
xml_df = list()
json_df = list()
date_format = '%Y-%m-%d'
for file in file_name:
    if file.endswith(".csv"): #if the file is .csv file
        df_csv = pd.read_csv(os.path.join('./data_to_stage',file)) #pd.read_csv('abc.csv')
        csv_df.append(df_csv)  
    if file.endswith(".xml"): #if the file is .xml file 
        df_xml = pdx.read_xml(os.path.join('./data_to_stage',file), ['ORDER', 'ORDERPOSITIONS'])
        xml_df.append(df_xml)
    if file.endswith(".json"): #if the file is .json file
        df_json = pd.read_json(os.path.join('./data_to_stage',file),orient='index')
#         df1 = df_json.drop('ORDERPOSITIONS',axis=1)
#         df2 = json_normalize(df_json['ORDERPOSITIONS'])
#         df = df1.merge(df2,left_index=True,right_index=True)
#         res = pd.json_normalize(df_json)
        json_df.append(df_json)

# print(json_df[0])
print(xml_df[0])
new = pd.DataFrame(xml_df[0])
print(new)

# #change date_time format
def convert_formats_to_datetimes(col, formats):
    out = [pd.to_datetime(col, format=x, errors='coerce',utc= True).dt.date for x in formats]
    return reduce(lambda l,r: pd.Series.combine_first(l,r), out)
# print(csv_df[1]['ORDERDATE'])
formats = ['%a, %d %b, %Y','%Y-%m-%d','%a,%d/%m/%y,%I:%M%p','%B %d, %Y' ,'%Y-%m-%dT%H:%M:%S%z']#but this one is not dynamic way.

csv_df[1]['ORDERDATE'] = csv_df[1]['ORDERDATE'].pipe(convert_formats_to_datetimes, formats)
# print(csv_df[1]['ORDERDATE'])


# FIELDS=['POSTID','ORDID','ARTID','QUANTITY','TOTALPRICE']
# def clean_data(data):
#     table = pd.DataFrame()
#     for i in range(len(data) - 1):
#         df = pd.json_normalize(data[i + 1])
#         df = df[FIELDS]
#         table = table.append(df)
#     return table 
csv_df[1]['ORDERDATE']

                                       ORDERPOSITION
0  {'@POSID': '1', 'QUANTITY': '11', 'ARTID': '20...
1  {'@POSID': '2', 'QUANTITY': '12', 'ARTID': '50...
2  {'@POSID': '3', 'QUANTITY': '13', 'ARTID': '20...
3  {'@POSID': '4', 'QUANTITY': '8', 'ARTID': '100...
                                       ORDERPOSITION
0  {'@POSID': '1', 'QUANTITY': '11', 'ARTID': '20...
1  {'@POSID': '2', 'QUANTITY': '12', 'ARTID': '50...
2  {'@POSID': '3', 'QUANTITY': '13', 'ARTID': '20...
3  {'@POSID': '4', 'QUANTITY': '8', 'ARTID': '100...


0     2002-01-02
1     2002-01-03
2     2002-01-03
3     2002-01-03
4     2002-01-03
5     2002-01-03
6     2002-01-03
7     2002-01-03
8     2002-01-03
9     2002-01-03
10    2002-01-03
11    2002-01-03
12    2002-01-03
13    2002-01-04
14    2002-01-04
15    2002-01-04
16    2002-01-04
17    2002-01-04
18    2002-01-04
19    2002-01-04
20    2002-01-04
21    2002-01-04
22    2002-01-04
23    2002-01-05
24    2002-01-05
25    2002-01-05
26    2002-01-05
27    2002-01-05
28    2002-01-05
29    2002-01-05
30    2002-01-05
31    2002-01-05
32    2002-01-05
33    2002-01-05
34    2002-01-05
35    2002-01-05
Name: ORDERDATE, dtype: object

Please ensure that field formats from the files match the given field formats from the database tables before enhancing the database tables with the new data from the files.
*(Hint: There are many useful libraries for handling csv, xml, json and datetime)*

In [21]:
## TODO#1:
#  Get creative on how you want to preprocess the data in the different files.
# print(json_df[0])
# result = json_normalize(json_df[0])
# print(json_df[0]['CUSTID'])
# print(json_df[0]['ORDERPOSITIONS.QUANTITY'])
# from pandas.io.json import json_normalize

# data = json_df[0]
# FIELDS=['POSTID','ORDID','ARTID','QUANTITY','TOTALPRICE']
# pd.json_normalize(data)
# for i in range(len(data) - 1):
#     df = pd.json_normalize(data[i + 1])
#     df = df[FIELDS]
#     table = table.append(df)
# result = json_normalize(data, 'ORDERPOSITION', FIELDS)
# result = json_normalize(data)
# result

### <font color='#782769'>Now it's your turn!<br>TODO #2:</font>

In [4]:
## Maybe you need to install the PostgreSQL database adapter first:
%pip install --upgrade pip
%pip install psycopg2
#or on unix: !sudo apt install python3-psycopg2

Requirement already up-to-date: pip in c:\users\schueler\appdata\local\programs\python\python38\lib\site-packages (20.2.4)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


After proprocessing our data, we need a script to INSERT the data into our existing tables in the Postgres database.

For that we first establish the connection and then write a loop which fills in the preprocessed data into an INSERT statement. This statement will then be sent to and executed by our database system.

In [37]:
import psycopg2

## Establishing the connection to your database
conn = psycopg2.connect(host='localhost',  # since you installed your PostgresDB locally 
                        port=5432,         # default PostgresDB port
                        user='postgres',   # the db user you used for database creation
                        password='1261993',       # TODO#2.1: Fill in the db password you used for database creation
                        database='bikeDB') # the name of the database you created

#Thanks to https://naysan.ca/2019/11/02/from-pandas-dataframe-to-sql-table-using-psycopg2/
def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# Inserting each row
for i in csv_df[0].index:
    print(csv_df[0]['posid'][i])
    query = """
            INSERT INTO articlereservation(posid, artid, artcount)
            VALUES(%s, %s, %s)""" %(csv_df[0]['posid'][i],csv_df[0]['artid'][i],csv_df[0]['artcount'][i])
    single_insert(conn, query)
for i in csv_df[1].index:
    # add '%s'::date for inserting date from dataframe to sql
    query = """
            INSERT INTO orders(ordid, orderdate, custid, staffid)
            VALUES(%s, '%s'::date, %s, %s)""" %(csv_df[1]['ORDID'][i],csv_df[1]['ORDERDATE'][i],csv_df[1]['CUSTID'][i],csv_df[1]['STAFFID'][i])
    query_2 = """INSERT INTO orderposition(posid, ordid, artid, quantity, totalprice)
                 VALUES(%s, %s, %s, %s, %s)"""%(csv_df[1]['POSID'][i],csv_df[1]['ORDID'][i],csv_df[1]['ARTID'][i],
                                               csv_df[1]['QUANTITY'][i],csv_df[1]['TOTALPRICE'][i])
    single_insert(conn, query)
    single_insert(conn, query_2)

## TODO#2.2: Write the code for inserting the prepared data into the database
#  <table_name> has to be replaced with the table you want to insert into
#  you can state the order of the column names at (<col_name1, ...)
#  and the (v1, v2, ...) has to be replaced with your preprocessed data
#  Note, that you also have to add %s into the string to match your number of arguments



# conn.commit() # <- We MUST commit to reflect the inserted data

# cursor.close()

# when a databse or cursor connection has been opened it also needs to be closed again
conn.close()


302
Error: duplicate key value violates unique constraint "pk_articlereservation"
DETAIL:  Key (posid, artid)=(302, 500002) already exists.

201
Error: duplicate key value violates unique constraint "pk_articlereservation"
DETAIL:  Key (posid, artid)=(201, 100002) already exists.

402
Error: duplicate key value violates unique constraint "pk_articlereservation"
DETAIL:  Key (posid, artid)=(402, 500001) already exists.

Error: duplicate key value violates unique constraint "pk_order"
DETAIL:  Key (ordid)=(14) already exists.

Error: duplicate key value violates unique constraint "pk_order"
DETAIL:  Key (ordid)=(15) already exists.

Error: duplicate key value violates unique constraint "pk_order"
DETAIL:  Key (ordid)=(16) already exists.

Error: duplicate key value violates unique constraint "pk_order"
DETAIL:  Key (ordid)=(16) already exists.

Error: duplicate key value violates unique constraint "pk_order"
DETAIL:  Key (ordid)=(16) already exists.

Error: duplicate key value violates u

If everything works fine, you should now have a database with 50000 orderposition entries, you can check that by right-clicking on the table in pgAdmin and selecting "Count Rows". A small alert in the bottom right corner pops up to tell you the number of entries! Or you can of course use:

```sql
SELECT COUNT(*) FROM public.orderposition
```

## Step 3 - Querying a Relational Database

### <font color='#782769'><br>Now it's your turn!<br>TODO #3:</font>
### <font color='#e52425'>This will be graded. You can reach a total of 2+2+2=6 points</font>

As soon as you tell your boss you integrated all the data, he asks you some questions.

Write an SQL Query for every question your boss askes you. You can test them in pgAdmin and copy the final version into this notebook.

```sql
-- TODO#3.1 (2 points): "Show me all customers, that are situated in Munich!"

SELECT *
FROM customer
WHERE place = 'Munich'



```

```sql
-- TODO#3.2 (2 points): "What is (are) the most expensive bike(s) that we shipped out?"

SELECT aid, name 
FROM article
WHERE aid = (SELECT aid FROM shipment WHERE netprice = (SELECT MAX(netprice) FROM shipment))



```

```sql
-- TODO#3.3 (2 points): "What do customers pay on average? (Take the average price over all orders.)"
#avg price of all customers pay
SELECT AVG(totalprice)
FROM orderposition
#avg price one customer pays
SELECT avg(orderposition.totalprice), customer.cid, customer.name
FROM orderposition
LEFT JOIN orders
ON orderposition.ordid = orders.ordid
INNER JOIN customer
ON orders.custid = customer.cid
GROUP BY customer.cid

```

## Step 4 - Creating a Data Warehouse

Just as you thought you were finally done with the database, your boss comes back from a big Buzzword-Conference and tells you, that he heard about Data Warehouses and that they are way quicker and cooler than relational databases! You could get even more insights as they have cool features like putting data in a cube or something... (He didn't quite got that part).

So, of course, he wants you to build one of these Warehouses now. 

In the file **CREATEwarehouse.sql** you will find the DDL-CREATE statements for the star-structure our data warehouse will have. (Check out the lecture for the theoretical part on that.) Again, right click on you bikeDB and select CREATE SCRIPT and copy the statements or open the file to add these four tables to your database. (Please ADD them to your existing schema, as we still need the old tables to fill the new ones!)

Make yourself familiar with the new structure. You can also create a small ER diagram on paper around the Facttable (like in the lecture shortly mentioned) for a better understanding.

Next, we need to fill the warehouse with our data. For that, we gave you a file called **procedure.sql**. This procedure will copy the data from the old tables into our new warehouse schema. 

In pgAdmin look for the "Procedures" tab under "Schemas" in your bikeDB. It should look pretty much like this (except you don't have a procedure created yet):
<img src="media/Screenshot_pgAdmin3.png" height=600>
Now, right-click on "Procedures" and create a new one. Give it a name like "fill-warehouse" and select "plpgsql" as "Language" in the "Definition" Tab. Then switch to the "Code" Tab and copy the code from the **procedure.sql** file and save your procedure.

Now it should show up in the list on the left. As we just created the procedure, we also need to execute it. For that right-click on the "fill_warehouse" procedure and select "Scripts > EXEC Script" or you can use 
```sql
CALL public.fill_warehouse()
```
in the Query Editor. Now check if your new tables are filled with data by either counting rows or querying the first few rows!

## Step 5 - Querying a Data Warehouse

Queries for data warehouses are not that different from the SQL queries you are used to. For one, they usually perform quicker on huge amount of data (which is hard to show here^^) because they avoid joins and they can be used to get a more general overview or a more detailed look into certain aspects of the data.

### <font color='#782769'>Now it's your turn!<br>TODO #4:</font>

Write an SQL query which only uses your Facttable and the new warehouse-tables (dcustomer, ddate and dproduct), which will show you all sold equipment between the week 201047 (week 47 of 2010) and 201050 per equipment and week.

```sql
--- TODO#4:

SELECT  COALESCE(TO_CHAR(P.ArtId, '999999'), '-SUM-') As ArtID, 
        COALESCE(P.Name, '---SUM---') As Name,
        COALESCE(TO_CHAR(WeekInYear, '999999'), 'ALLWEEKS') As Week, 
        COALESCE(SUM(F.Quantity),0) AS Quantity
FROM    Facttab F -- JOIN ...
--      ...
--      ...
ORDER BY 1 ASC, 3 ASC;
```

Your result should look like this:
<img src="media/WH.png" height=600>

## Step 6 - Setting up a NoSQL Database

Even though your boss is very happy with the new warehouse and all the information you can get from it with just a few lines of code, he still insists that there's got to be an even quicker way... He also tells you, that he read a little into data warehouses (so now he is of course an expert on it -.-) and figured out that data warehouses are "sooooo 1980s" and "the company needs to stay fresh and keep up with the trends..." (You also kinda expect him to use either the term Machine Learning of Artificial Intelligence next but he doesn't...yet.)

So, of course you offer to migrate at least a part of your database to a NoSQL database. NoSQL is more suitable for flexible and expendable environments and is therefore used in companies that deal with a lot of (possibly changing) data, questionable for your bike shop, but hey, the boss said so... ;)

First, we need to set up (another) database environment. We are gonna use MongoDB for this example, which is one of the most frequent used, document-oriented database management systems. Please download the Community Version from [their website](https://www.mongodb.com/try/download/community) and install it on your computer. After installing, you might need to set the PATH variables in Windows. You can come ask us, if you have any troubles with that.

If you are working on a Unix-System, you can find a detailed installation guide in the **mongoDB_on_Linus.txt** file.

On Windows, mongoDB will automatically install "mongoDB Compass" which is a GUI for NoSQL databases. There are also a ton of tools, which help you to migrate a relational database into a non-relational document-oriented filesystem. If you are interested, here are a couple of useful links for further reading:
* https://www.mongodb.com/try/download/compass # IDE for mongodb and migration tool for RDBMS to mongodb 
* https://robomongo.org/ (Robo 3T) # IDE for mongodb and migration tool for RDBMS to mongodb
* https://github.com/gagoyal01/mongodb-rdbms-sync # migration tool for RDBMS to mongodb
* https://github.com/compose/transporter # migration tool for RDBMS to mongodb 
* https://www.researchgate.net/project/Darwin-Schema-Management-in-NoSQL-Databases # migration tool for RDBMS to mongodb 

In order to learn what the tools are doing you will implement the migration partially from scratch.

Therefore, the following lines show you the setup and basic functionalities of the mongodb adapter pymongo:

In [6]:
### install mongodb adapter pymongo:
%pip install pymongo --no-cache

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [3]:
## First, we declare(!) a new database called bikeDB
#  We can also check if the connection works (connect=True)

db = client.bikeDB
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bikeDB')

In [4]:
## Next, we declare(!) a collection of files (called article)
#  Remember, NoSQL in general is document based, so we are working on 
#  a number of different files not on tables anymore.
#  So what used to be tables are now collections.

collection = db.article
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bikeDB'), 'article')

In [15]:
## Let's take a look at what is inside of the collection.

db.list_collection_names()

[]

As you can see, there is nothing. Why is that the case?
An important note about collections (and databases) in MongoDB is that they are created lazily - none of the above commands have actually performed any operations on the MongoDB server. Collections and databases are created when the first document is inserted into them.
So let's do that next.

In [16]:
lst_collection_documents = []

## As a Reminder, the columns of old table article: 
#    aid, name, net, tax, price, color, description, measure, made

#  This is a sample document
dict_article_document = {
    #"_id": 100001, # sets e.g. aid as documentid instead of the auto-created ObjectId("5fb7000f0a2b90a2af21f7a8")
    "aid": 100001,
    "name": "Man City Bike",
    "net": 588.24,
    "tax": 111.76,
    "price": 700.00,
    "color": 'blue',
    "description": '26 inch',
    "measure": 'P',
    "made": 'E'
}

#  Declares collection article, same as above
article_collection = db.article 

#  Now we collect the created document(s) for bulk insert in list
lst_collection_documents += [dict_article_document] 

## Adds all documents dict_article_document (row/tuple) from list lst_collection_documents 
#  to collection article_collection via bulk insert command "insert_many"
result = article_collection.insert_many( lst_collection_documents ) 
result.inserted_ids

[ObjectId('5fbd13e735c6ccf8ff97eb2a')]

In [17]:
## Now after the first document insertion the collection article as well as 
#  the database bikeDB get visible 
print(db.list_collection_names())
for dba in client.list_databases():
    print(f"db: {dba['name']}")

['article']
db: admin
db: bikeDB
db: config
db: local


### <font color='#782769'>Now it's your turn!<br>TODO #5:</font>
### <font color='#e52425'>This will be graded. You can reach a total of 1+2+1=4 points</font>

Below, you can see a prepared script to add the collections for *orders*, *orderposition*, *staff* and *article* into the MongoDB database. First we load the data from the files, you can find in the folder called **data_nosql** then we declare the collections.

In [7]:
## These are the files that need to be imported

import os
data_path = './data_nosql'
os.listdir(data_path)

['customer.csv', 'article.csv', 'orderposition.csv', 'staff.csv', 'orders.csv']

The declaration for the collection for *customer* is missing. Try to understand the script and add the *customer* collection declaration.

Further down in the *get_collection_document_list* function, there is also a loop missing which iterativly created your documents and adds them to the list we want to insert. 

In [11]:
## hint: to lookup different kinds of table relationships see http://learnmongodbthehardway.com/schema/schemabasics/

import pandas as pd
   
def load_bikedb_rdbms_data():
    
    df_article = pd.read_csv(f"{data_path}/article.csv", sep=';')
    ## TODO#5.1 (1 point): Load the csv file for the customers
    df_customers = pd.read_csv(f"{data_path}/customer.csv", sep=';')
    df_orderposition = pd.read_csv(f"{data_path}/orderposition.csv", sep=';')
    df_orders = pd.read_csv(f"{data_path}/orders.csv", sep=';')
    df_staff = pd.read_csv(f"{data_path}/staff.csv", sep=';')
    
    return {
        "orders": {
            "df": df_orders,
            "pk": 'ordid',
            "fks": [
                {
                    "df_fk_table": df_orderposition,
                    "fk_colname": 'ordid',
                    "fk_pk_colname": 'posid'        
                }
            ]
        },
        "orderposition": {
            "df": df_orderposition,
            "pk": 'posid',
            "fks": []
        },

        ## TODO#5.2 (2 points): Add the customer collection here:
        # "customer: {...},"
      "customer": {
            "df": df_customers,
            "pk": 'cid',
            "fks": [
                {
                    "df_fk_table": df_orders,
                    "fk_colname": 'custid',
                    "fk_pk_colname": 'ordid'        
                }
            ]
        },
                
        
        "staff": {
            "df": df_staff,
            "pk": 'staffid',
            "fks": [
                {
                    "df_fk_table": df_orders,
                    "fk_colname": 'staffid',
                    "fk_pk_colname": 'ordid'        
                }
            ]
        },
        "article": {
            "df": df_article,
            "pk": 'aid',
            "fks": [
                {
                    "df_fk_table": df_orderposition,
                    "fk_colname": 'artid',
                    "fk_pk_colname": 'posid'        
                }
            ]
        }
    }

def lookup_fks(df_fk_table, fk_colname, fk_value, fk_pk_colname):
    
    return list(set(df_fk_table.query(f"{fk_colname} == {fk_value}")[fk_pk_colname].tolist()))
    

def create_document(row, pk_colname, doc_id_field=None, fk_relations=[]):
    dict_row = row
    
    # Ensure type dict
    if type(row) != dict:
        dict_row = row.to_dict()
    
    # Create foreign key list fields
    for fk in fk_relations:
        dict_row[f"{fk['fk_pk_colname']}s"] = lookup_fks(
            fk['df_fk_table'], fk['fk_colname'], dict_row[pk_colname], fk['fk_pk_colname'])
    
    # Ensure that value of doc_id_field to _id document identifier (primary key)
    if doc_id_field:
        return {'_id': int(dict_row[doc_id_field]), **dict_row}
    return {**dict_row}

def get_collection_document_list(data):
    
    collection_document_list = []

    ## TODO#5.3 (1 point): Write a loop that for each row in your dataframe,
    #  creates a document and add that documents to the list defined above. 
    
    #for ...
    
    
        
    return collection_document_list

def migrate_bikedb_from_rdbms_to_mongodb():
    
    client = MongoClient('localhost', 27017)
    db = client.bikeDB
    
    rdbms_data = load_bikedb_rdbms_data()
    
    
    for table, data in rdbms_data.items():
        print(f" ---- {table} ----")
        
        # clean before
        collection = db[table]
        collection.drop() 
        
        # create collection content
        collection = db[table]
        collection_document_list = get_collection_document_list(data)
        
        # write collection content to mongodb
        result = collection.insert_many(collection_document_list)         
        print(result.inserted_ids)
        

migrate_bikedb_from_rdbms_to_mongodb()

 ---- orders ----


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 5fcd963fab929bfd3384c713, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

If everything worked out, you should now see a long list of all documents inserted into your NoSQL database!

## Step 7 - Querying a NoSQL Database

After you have now successfully migrated the bikeDB table selection to mongodb, your boss wants to have the same reports available as before. That way, you could theoretically also compare, if NoSQL databases are indeed quicker and "cooler".

As an example, let's say we want to find all article IDs and names with a price higher than 600.0. In SQL this query would look like this:
```sql
SELECT aid, name
FROM article
WHERE price > 600.0;
```

When querying with NoSQL however, the query would be written like this:

In [27]:
for r in db.article.find({"price": {"$gt": 600.0}}):
    print(r["_id"], r["name"])

100001 Man City Bike
100002 Woman City Bike
100003 Man City Bike
100004 Woman City Bike
100005 Man City Bike
100006 Woman City Bike
100007 Man City Bike
100008 Woman City Bike
100011 Man Trekking Bike
100012 Woman Trekking Bike
100013 Man Trekking Bike
100014 Woman Trekking Bike
100015 Man Trekking Bike
100016 Woman Trekking Bike
100017 Man Trekking Bike
100018 Woman Trekking Bike
100021 Mountainbike
100022 Mountainbike
100023 Mountainbike


A simple and great overview of the different commands used in NoSQL queries [can be found here](https://info-mongodb-com.s3.amazonaws.com/ReferenceCards15-PDF.pdf)!

### <font color='#782769'>Now it's your turn!<br>TODO #6:</font>

In [29]:
## Redo the first SQL Query from TODO#3.1 in NoSQL:
#  "Show me all customers, that are situated in Munich!"
#  And compare your results.



